<a href="https://colab.research.google.com/github/aeyjeyaryan/Deep-Learning/blob/main/TinyLlama_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs and Imports


In [1]:
!pip install torch datasets transformers peft --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00


In [2]:
!pip install bitsandbytes --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.9 MB/s eta 0:00:00


In [3]:
!pip install datasets --q

In [4]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

In [5]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [10]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

## Load Data

In [42]:
import json
from itertools import chain
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM

with open("conversations.json", "r") as f:
    data = json.load(f)
for i, item in enumerate(data):
    assert 'instruction' in item and 'response' in item, f"Missing keys at idx {i}"

dataset = Dataset.from_list(data)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['instruction'], batch['response'])
    ]
    tokens = tokenizer(texts,
                       padding='max_length',
                       truncation=True,
                       max_length=256)
    tokens['labels'] = [
        [(t if t != tokenizer.pad_token_id else -100) for t in ids]
        for ids in tokens['input_ids']
    ]
    return tokens

tokenized = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)

vocab_size = len(tokenizer)
all_input_ids = list(chain.from_iterable(tokenized["input_ids"]))
all_labels    = list(chain.from_iterable(tokenized["labels"]))
assert all(0 <= t < vocab_size for t in all_input_ids),    "Input ID OOB"
assert all(t == -100 or 0 <= t < vocab_size for t in all_labels), "Label ID OOB"
print("✅ Tokenization OK")

batch_size = 4
def collate_fn(batch):
    input_ids = torch.tensor([ex["input_ids"] for ex in batch], dtype=torch.long)
    attention_mask = torch.tensor([ex["attention_mask"] for ex in batch], dtype=torch.long)
    labels = torch.tensor([ex["labels"] for ex in batch], dtype=torch.long)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

loader = DataLoader(tokenized, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 3
for epoch in range(num_epochs):
    total_loss = 0.0
    for step, batch in enumerate(loader):
        # Move to device
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss / 4
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % 4 == 0:
            optimizer.step()
            optimizer.zero_grad()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}/{num_epochs} — avg loss: {avg_loss:.4f}")

model.save_pretrained("./training_output")
tokenizer.save_pretrained("./training_output")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Tokenization OK
Epoch 1/3 — avg loss: 0.9614
Epoch 2/3 — avg loss: 0.5648
Epoch 3/3 — avg loss: 0.3493


('./training_output/tokenizer_config.json',
 './training_output/special_tokens_map.json',
 './training_output/vocab.json',
 './training_output/merges.txt',
 './training_output/added_tokens.json',
 './training_output/tokenizer.json')

In [46]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_dir = "./training_output"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForCausalLM.from_pretrained(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

print("Chat with your model (type 'exit' or 'q' to quit)")
while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ("exit", "q"):
        print("Exiting chat…")
        break

    prompt = f"### Instruction:\n{user_input}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_length=inputs["input_ids"].shape[1] + 128,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # strip everything up through the first marker ...
    part = text.split("### Response:\n", 1)[-1]
    # ...then drop any extra repeats
    reply = part.split("### Response:")[0].strip()

    print(f"Model: {reply}\n")


Chat with your model (type 'exit' or 'q' to quit)
You: Summarize my profile based on past conversations.
Model: I'm a tech entrepreneur who focuses on empowering people through apps, projects, and business solutions. [v7]

You: Suggest a business model for EcoExchange.
Model: Create a platform where users can submit and track issues and track progress. [v4]

You: q
Exiting chat…
